
# Week 01 Assignment  
## Data Quality, Evaluation, Scaling, and Encoding

**Student name:Abdul Hakim Akash**   

This is a small assignment that connects topics from Module 1, 2, and 3.  
You must complete it in this Colab notebook.

You will need to use concepts that appeared in the videos:
- Module 1 and 2: basic descriptive statistics, proportions, confusion matrix, accuracy, precision, recall
- Module 3: standardization, min max scaling, nominal vs ordinal, one hot encoding, ordinal encoding, Euclidean and Manhattan distance

Please do not use any extra libraries beyond `pandas`, `numpy`.



---
## 0. Setup and Dataset

We will use a dataset that should have columns given below:

- `user_id`  
- `age`  
- `monthly_income` (numeric)  
- `daily_screen_time_min` (numeric)  
- `daily_app_opens` (numeric)  
- `true_label` and `pred_label` for a binary classification task (0 or 1)  
- `satisfaction_level` (for example: `Low`, `Medium`, `High`)  
- `city_type` (for example: `Urban`, `Suburban`, `Rural`)


In [ ]:
# Cell 1: Imports
import pandas as pd
import numpy as np

In [ ]:
# Cell 2: Load the dataset (Already done for you)
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1OmDDCh4MD1TtvAemnwVDyz5zwCIXJ220")

# Show first few rows
df.head()

,user_id,age,monthly_income,daily_screen_time_min,daily_app_opens,true_label,pred_label,satisfaction_level,city_type
0,1,43,3734.19,109,48,0,0,Medium,Suburban
1,2,49,2594.19,194,7,0,0,Low,Urban
2,3,19,3550.47,146,36,1,0,High,Rural
3,4,19,3821.18,287,14,1,0,High,Suburban
4,5,63,1750.84,66,46,0,0,Medium,Suburban



### 0.1 Check your dataset

1. Confirm that the dataset loaded correctly.  
2. Check that you have at least these columns:  
   - numeric: `age`, `monthly_income`, `daily_screen_time_min`, `daily_app_opens`  
   - labels: `true_label`, `pred_label`  
   - categorical: `satisfaction_level`, `city_type`  



---
## Part A - Module 1 and 2 Review

In this part you will do simple descriptive statistics and basic classification evaluation.



### Q1. Descriptive statistics on a numeric feature

Choose one numeric column, for example `daily_screen_time_min`.


In [ ]:
# Q1.1: Choose your numeric column here [We already write this ans]
num_col = "daily_screen_time_min"

df[num_col].describe()

,daily_screen_time_min
count,100.000000
mean,181.890000
std,68.886951
min,60.000000
25%,122.000000
50%,178.000000
75%,243.750000
max,299.000000



> **Q1.2 Short answer: [Marks: 05]**  
> Look at the count, mean, min, max, and standard deviation for your chosen column.  
> In 2 to 3 sentences, comment on what you see.  
> For example, does the max look very far from the mean, or does it look quite close?

Write your answer here:

>  The values in the daily_screen_time_min column range from 60 to 299 minutes, with a mean of about 182 minutes. The maximum value indicates a moderate dispersion rather than extreme outliers because it is slightly higher than the mean but not very far. Users' screen time varies significantly, as evidenced by the standard variance of almost 69 minutes.
>  
>  



### Q2. Proportion of positive class

Use the `true_label` column, where 1 means "positive" and 0 means "negative".


In [ ]:
# Q2.1: Compute proportion of positive class [We already write this ans]
label_col = "true_label"

positive_count = (df[label_col] == 1).sum()
total_count = df.shape[0]
positive_proportion = positive_count / total_count

print("Positive count:", positive_count)
print("Total samples:", total_count)
print("Proportion of positive class:", positive_proportion)

Positive count: 52
Total samples: 100
Proportion of positive class: 0.52



> **Q2.2 Short answer: [5 marks]**  
> In 1 to 2 sentences, explain what this proportion tells you about your dataset.  
> For example, is the dataset balanced between 0 and 1, or is one class much more common?

Write your answer here:

>  The dataset is nearly evenly distributed between class 0 and class 1, as indicated by the positive class proportion of 0.52. Class imbalance is not a significant issue here because neither class predominates in the sample.
>  
>  



### Q3. Confusion matrix and basic metrics

For this question, use:
- `true_label` as the actual label  
- `pred_label` as the model prediction


In [ ]:
# Q3.1: Manually compute TP, TN, FP, FN [We already write this ans]
true_col = "true_label"
pred_col = "pred_label"

tp = ((df[true_col] == 1) & (df[pred_col] == 1)).sum()
tn = ((df[true_col] == 0) & (df[pred_col] == 0)).sum()
fp = ((df[true_col] == 0) & (df[pred_col] == 1)).sum()
fn = ((df[true_col] == 1) & (df[pred_col] == 0)).sum()

print("TP:", tp)
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)

TP: 28
TN: 27
FP: 21
FN: 24


In [ ]:
# Q3.2: Compute accuracy, precision, recall [We already write this ans]
accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.55
Precision: 0.5714285714285714
Recall: 0.5384615384615384



> **Q3.3 Short answer: [10 marks]**  
> In 3 to 4 sentences, briefly comment on the model using these three metrics.  
> For example, is the model catching most positives (high recall) or being careful when it predicts positive (high precision)?

Write your answer here:

>  The model has an accuracy of 0.55, which means it correctly predicts slightly more than half of the cases. The precision is 0.57, indicating that when the model predicts a positive, it is correct about 57% of the time. The recall is 0.54, showing that the model identifies just over half of the actual positives. Overall, the model is moderately balanced between catching positives and avoiding false positives, but its performance could be improved.
>  
>  



---
## Part B - Module 3: Scaling and Encoding

Now we will pick a few features and apply scaling and encoding.



### Q4. Standardization and Min max scaling

Use one numeric column, `monthly_income`.


In [ ]:
# Q4.1: Choose the numeric column [2 marks]
m_income = df['monthly_income']
m_income.head()


,monthly_income
0,3734.19
1,2594.19
2,3550.47
3,3821.18
4,1750.84


In [ ]:
#print(df['monthly_income'].isna().sum())

In [ ]:
# Q4.2: Standardization with z-score [10 marks]
mean = m_income.mean()
std = m_income.std()
z_scores = (m_income - mean) / std
print(z_scores.head(20))

0     0.944685
1    -0.324626
2     0.740126
3     1.041542
4    -1.263639
5     1.288969
6     1.325846
7     0.084192
8    -0.518464
9    -2.099647
10   -0.412699
11    0.151711
12   -0.823065
13   -0.529487
14    1.036710
15   -1.759182
16    1.187847
17    0.775499
18    1.311806
19   -0.282182
Name: monthly_income, dtype: float64


In [ ]:
# Q4.3: Min max scaling implementation [10 marks]
min_value = m_income.min()
max_value = m_income.max()
min_max_scaled = (m_income - min_value) / (max_value - min_value)
print(min_max_scaled.head(20))

0     0.675209
1     0.393685
2     0.629839
3     0.696691
4     0.185420
5     0.751568
6     0.759747
7     0.484358
8     0.350694
9     0.000000
10    0.374152
11    0.499333
12    0.283136
13    0.348249
14    0.695619
15    0.075512
16    0.729140
17    0.637685
18    0.756633
19    0.403099
Name: monthly_income, dtype: float64



> **Q4.4 Short answer: [3 marks]**  
> Compare the standardized and min max scaled columns in 2 to 3 sentences.  
> Mention what kind of range each one uses and how the numbers look.

Write your answer here:

>  The data is transformed by standardization (z-score) to have a mean of zero and a standard deviation of one. This indicates that the values are centered around zero and can be either positive or negative based on how much they deviate from the mean.
>  Min-max scaling rescales all values into the range 0 to 1, so every number becomes a small decimal and there are no negative values.
>  Overall, both methods help make the feature values comparable by putting them on a similar scale, which improves how models understand the relationships between data points.



### Q5. One hot and ordinal encoding

We will use:
- `city_type` as a nominal feature  
- `satisfaction_level` as an ordinal feature with order `Low` < `Medium` < `High`  


In [ ]:
df.head()

,user_id,age,monthly_income,daily_screen_time_min,daily_app_opens,true_label,pred_label,satisfaction_level,city_type
0,1,43,3734.19,109,48,0,0,Medium,Suburban
1,2,49,2594.19,194,7,0,0,Low,Urban
2,3,19,3550.47,146,36,1,0,High,Rural
3,4,19,3821.18,287,14,1,0,High,Suburban
4,5,63,1750.84,66,46,0,0,Medium,Suburban


In [ ]:
# Q5.1: One hot encoding for city_type using pandas [10 marks]
city_dummies = pd.get_dummies(df['city_type'], prefix='city', dtype=int)
city_dummies.head()

,city_Rural,city_Suburban,city_Urban
0,0,1,0
1,0,0,1
2,1,0,0
3,0,1,0
4,0,1,0


In [ ]:
# Q5.2: Attach one hot encoded columns to df [5 marks]
df = pd.concat([df, city_dummies], axis=1)
df.head()

,user_id,age,monthly_income,daily_screen_time_min,daily_app_opens,true_label,pred_label,satisfaction_level,city_type,city_Rural,city_Suburban,city_Urban
0,1,43,3734.19,109,48,0,0,Medium,Suburban,0,1,0
1,2,49,2594.19,194,7,0,0,Low,Urban,0,0,1
2,3,19,3550.47,146,36,1,0,High,Rural,1,0,0
3,4,19,3821.18,287,14,1,0,High,Suburban,0,1,0
4,5,63,1750.84,66,46,0,0,Medium,Suburban,0,1,0


In [ ]:
# Q5.3: Ordinal encoding for satisfaction_level [10 marks]
order = {'Low': 1, 'Medium': 2, 'High': 3}
df['satisfaction_level'] = df['satisfaction_level'].map(order).astype(int)
df.head()

,user_id,age,monthly_income,daily_screen_time_min,daily_app_opens,true_label,pred_label,satisfaction_level,city_type,city_Rural,city_Suburban,city_Urban
0,1,43,3734.19,109,48,0,0,2,Suburban,0,1,0
1,2,49,2594.19,194,7,0,0,1,Urban,0,0,1
2,3,19,3550.47,146,36,1,0,3,Rural,1,0,0
3,4,19,3821.18,287,14,1,0,3,Suburban,0,1,0
4,5,63,1750.84,66,46,0,0,2,Suburban,0,1,0



> **Q5.4 Short answer: [5 marks]**  
> In 2 to 3 sentences, explain why one hot encoding is suitable for `city_type`  
> and why ordinal encoding is suitable for `satisfaction_level`.

Write your answer here:

>  Because city_type is a categorical variable with no inherent order, one-hot encoding works well for it. Additionally, by establishing distinct binary columns, the model is prevented from assuming any ranking between the categories.

>  Because the categories Low, Medium, and High have a meaningful order and mapping them to numbers (1, 2, 3) accurately retains this hierarchy for machine learning models, ordinal encoding is appropriate for satisfaction_level.
>  



---
## Part C - Module 3: Distances between users

For this small part we will work with vectors based on scaled numeric features.



### Q6. Euclidean and Manhattan distance

Build 2D vectors for user 0 and user 1 using:
- `income_std`  
- `daily_app_opens` (or its min max scaled version if you prefer)


In [ ]:
# Q6.1: Build 2D vectors for first two users [We already write this ans]
vec_cols = ["monthly_income", "daily_app_opens"]

v1 = df.loc[0, vec_cols].values
v2 = df.loc[1, vec_cols].values

print("v1:", v1)
print("v2:", v2)

v1: [np.float64(3734.19) np.int64(48)]
v2: [np.float64(2594.19) np.int64(7)]


In [ ]:
# Q6.2: Euclidean distance computation [5 marks]
ed = np.linalg.norm(v1 - v2)
print("Euclidean distance:", np.round(ed, 3).tolist())

Euclidean distance: 1140.737


In [ ]:
# Q6.3: Manhattan distance computation [5 marks]
ma = np.linalg.norm(v1 - v2, ord=1)
print("Manhattan distance:", ma.tolist())

Manhattan distance: 1181.0



> **Q6.4 Short answer: [5 marks]**  
> Which one is larger in your result, Euclidean or Manhattan distance  
> and why does that usually happen based on their formulas?

Write your answer here:

>  The Manhattan distance is larger than the Euclidean distance. This usually happens because Manhattan distance adds the absolute differences directly, while Euclidean distance takes the square root of the sum of squared differences, which reduces the final value.

>  In other words, Euclidean distance measures the straight-line path, while Manhattan distance measures a longer step by step path.
>  



---
## Final Reflection [10 marks]

> In 4 to 6 sentences, describe how the three modules connect in this assignment.  
> Mention:
> - One idea from Module 1 or 2 that you used  
> - One idea from Module 3 that you used  
> - How these ideas together help you understand a dataset more deeply

Write your reflection here:

>  In this assignment, Module 1 and 2 helped me explore the dataset using descriptive statistics, outlier detection, and probability-based evaluation like proportions and confusion matrix metrics. These techniques gave me a clear understanding of how the data is distributed and whether any values were unusual. From Module 3, I applied scaling methods, encoding methods, and distance calculations to prepare and compare the features properly. Scaling and encoding were important because they converted raw values into forms that machine learning models can understand. By combining statistical insights with proper preprocessing, I was able to analyze the dataset more deeply and see how different users or samples relate to each other.
>  
>  



## End of Assignment

Before submitting:
- Run all cells from top to bottom.  
- Check that all answer sections are filled.  
- Instruction video অনুযায়ী আমাদের দেয়া Colab ফাইলটি থেকে প্রথম একটি Save copy in drive করে নিবা। এরপর Google colab এর মধ্যে কোডগুলো করবে এবং সেই ফাইলটি ‘Anyone with the link’ & ‘View’ Access দিয়ে ফাইলটির Shareble Link টি সাবমিট করবে।
